## Creación y Entrenamiento de un Modelo de Detección de Objetos


### **Configurar el Entorno de Trabajo**

- **Instala las librerías necesarias**: Asegúrate de tener TensorFlow, Keras, Pandas, OpenCV y Matplotlib.
    
    ```bash
    pip install tensorflow keras pandas opencv-python matplotlib
    ```
    
    Estas librerías son las principales, más esenciales y básicas a la hora de trabajar con modelos de inteligencia artificial.
    
- **Configuración de GPU** (si tienes acceso): Aumentará la velocidad de entrenamiento significativamente. Configura TensorFlow para utilizar la GPU, si está disponible.

In [120]:
%pip install tensorflow keras pandas opencv-python matplotlib
%matplotlib inline

Note: you may need to restart the kernel to use updated packages.


### **Carga y Preparación del Dataset**

En este caso, se va a trabajar con un csv que contiene los datos del dataset

- **Leer el archivo CSV**: Utiliza Pandas para leer el archivo CSV que contiene los nombres de las imágenes, las coordenadas de las bounding boxes (cajas delimitadoras) y las clases de los objetos.

- **Estructura del CSV**: Asegúrate de que el archivo CSV contenga las columnas necesarias:

    - `filename`: Nombre de la imagen.

    - `xmin`, `ymin`, `xmax`, `ymax`: Coordenadas de las bounding boxes.

    - `class`: Etiqueta de la clase del objeto.

In [121]:
import pandas as pd

df_dataset = pd.read_csv("dataset/dataset.csv")

df_dataset.columns

Index(['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class'], dtype='object')

### **Preprocesamiento de Imágenes y Etiquetas**

- **Función de Carga de Imágenes**: Crea una función para cargar imágenes desde las rutas especificadas en el CSV y redimensionarlas a un tamaño consistente (por ejemplo, 416x416 para YOLO o 300x300 para SSD).

- **Normalización de Coordenadas**: Convierte las coordenadas de las bounding boxes a un formato normalizado (0-1) dividiendo entre el ancho y alto de la imagen.

- **Aumento de Datos (Data Augmentation)**: Aplica técnicas de aumento de datos, como rotaciones, recortes aleatorios, o ajustes de brillo y contraste, para aumentar la robustez del modelo.

In [122]:
import cv2

def loadImageData(df, image_dir, image_size):
    # Diccionario para almacenar las imágenes y las bounding boxes
    images, boxes, classes = [], [], []

    # Recorre el DataFrame
    for idx, row in df.iterrows():
        img_path = f"{image_dir}/{row['filename']}"
        image = cv2.imread(img_path)
        image = cv2.resize(image, (image_size, image_size))
        images.append(image / 255.0)  # Normalización de la imagen

        # Bounding box normalizada
        xmin = row['xmin'] / image.shape[1]
        ymin = row['ymin'] / image.shape[0]
        xmax = row['xmax'] / image.shape[1]
        ymax = row['ymax'] / image.shape[0]
        boxes.append([xmin, ymin, xmax, ymax])
        classes.append(row['class'])

    return images, boxes, classes

In [123]:
loadImageData(df=df_dataset, image_dir="dataset/data/train/plasticBottle", image_size=300)

([array([[[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
  
         [[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
  
         [[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
  
         ...,
  
         [[0.00392157, 0.00392157, 0.00392157],
          [0.0

### **Definir la Arquitectura del Modelo**

- **Seleccionar un Modelo Preentrenado**: Usar un modelo preentrenado y adaptarlo a tu dataset es una estrategia eficaz y optimizada. Por ejemplo, modelos como **YOLOv3**, **SSD**, o **Faster R-CNN** ya están bien optimizados para detección de objetos y solo requieren ajustar las capas finales para tu conjunto de clases.

- **Cargar un Backbone Preentrenado**: Puedes usar un modelo base preentrenado en COCO (por ejemplo, MobileNet o ResNet) y luego añadir capas para detección de objetos.

Aquí se muestra cómo cargar y personalizar un modelo SSD o YOLO utilizando Keras.

In [124]:
import tensorflow as tf

from keras import layers, models
from keras._tf_keras.keras.applications import MobileNetV2 

In [125]:
# Cargar un backbone como MobileNetV2

base_model = MobileNetV2(input_shape=(300, 300, 3), include_top=False)
base_model.trainable = False  # Congelar las capas

C:\Users\lruiz\AppData\Local\Temp\ipykernel_18404\681146479.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(300, 300, 3), include_top=False)


In [126]:
# Añadir capas de detección de objetos

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation="relu"),
    # Número de clases de tu dataset
    layers.Dense(1, activation="sigmoid")
])

In [127]:
print(model)

<Sequential name=sequential_5, built=True>


### **Definir la Pérdida y Métricas**

- **Función de Pérdida**: Usa una combinación de pérdida de regresión para las coordenadas de las bounding boxes (como `IoU`) y una pérdida de clasificación (`Categorical Crossentropy`) para la detección de clases.

- **Métricas de Evaluación**: Agrega métricas personalizadas, como `IoU` o precisión en las bounding boxes.

In [128]:
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

### **Entrenamiento del Modelo**

- **Generador de Datos**: Configura un generador de datos que lea el dataset en lotes (`batch`) y aplique el preprocesamiento y aumento de datos en tiempo real.

- **Ajuste de Hiperparámetros**: Define un tamaño de lote adecuado (como 16 o 32) y una tasa de aprendizaje inicial (como 0.001). Puedes utilizar técnicas como la reducción de tasa de aprendizaje en `plateau` para mejorar la convergencia.

Se va a utilizar el método `model.fit()` de Keras para entrenar un modelo, y configurando el callback `ReduceLROnPlateau`. 

Este callback se usa para reducir la tasa de aprendizaje cuando el modelo deja de mejorar en la métrica monitoreada.

In [129]:
from keras._tf_keras.keras.callbacks import ReduceLROnPlateau

# Configuración de ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',   # Métrica a monitorear
                                                    # Factor de reducción de la tasa de aprendizaje (se divide por 2 en este caso)
                              factor=0.5,
                              patience=5,           # Número de épocas sin mejora para reducir el LR
                              min_lr=1e-6)          # Límite mínimo para el LR

Para ejecutar este código de entrenamiento, debes definir tres elementos esenciales: `train_generator`, `val_generator` y `num_epochs`. Aquí te explico qué significa cada uno y cómo configurarlos:

#### `train_generator` y `val_generator`

`train_generator` y `val_generator` son generadores de datos, que generalmente se usan para cargar imágenes u otros datos de manera progresiva durante el entrenamiento. Esto es útil cuando trabajas con grandes cantidades de datos, como imágenes, que no caben en memoria. Estos generadores generan lotes de datos en cada iteración, lo que permite un entrenamiento eficiente.

#### Explicación de parámetros:

- **`target_size`**: Tamaño al que se redimensionarán las imágenes (ajústalo según lo que necesite tu modelo).

- **`batch_size`**: Tamaño de cada lote (32 en este caso).

- **`class_mode`**: Tipo de clasificación; `'binary'` para dos clases y `'categorical'` para varias clases.

#### Alternativa para otros tipos de datos

Si trabajas con datos tabulares, series temporales o texto, puedes usar `tf.data.Dataset` para crear generadores de datos específicos de cada caso.

In [130]:
import keras._tf_keras as ktf

from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator

# Definición del generador de imágenes con preprocesamiento básico
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,            # Escala los valores de los píxeles entre 0 y 1
    rotation_range=20,            # Aumentación: rotación
    width_shift_range=0.2,        # Aumentación: desplazamiento horizontal
    height_shift_range=0.2,       # Aumentación: desplazamiento vertical
    shear_range=0.2,              # Aumentación: cizalladura
    zoom_range=0.2,               # Aumentación: zoom
    horizontal_flip=True,         # Aumentación: voltear horizontalmente
    fill_mode='nearest'           # Relleno de píxeles vacíos tras transformación
)

# Generador de datos de entrenamiento
train_generator = train_datagen.flow_from_directory(
    './dataset/data/train/',  # Ruta a la carpeta de entrenamiento
    target_size=(300, 300),       # Tamaño de las imágenes que se generarán
    batch_size=32,                # Tamaño de cada lote
    class_mode='categorical'           # Tipo de clasificación: 'binary' o 'categorical'
)

# Generador de datos de validación (sin aumentación)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_generator = val_datagen.flow_from_directory(
    './dataset/data/test/',    # Ruta a la carpeta de validación
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

Found 14 images belonging to 1 classes.
Found 4 images belonging to 1 classes.


#### `num_epochs`

`num_epochs` es el número de veces que el modelo verá los datos completos de entrenamiento. 

Para comenzar, puedes usar un valor como `10`, `20`, o incluso `50`. Entrenar más épocas permite que el modelo aprenda más, pero puede llevar a un sobreajuste si el modelo empieza a aprender patrones específicos de los datos de entrenamiento sin mejorar en los datos de validación.

In [131]:
num_epochs = 50  # Número de épocas (ajústalo según tus datos)

In [132]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [133]:
# Entrenamiento del modelo

history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=num_epochs,
                    callbacks=[reduce_lr])

c:\Proyectos\IA\object_detection\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.4286 - loss: 0.7070 - val_accuracy: 1.0000 - val_loss: 0.0100 - learning_rate: 0.0010
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 585ms/step - accuracy: 1.0000 - loss: 0.0109 - val_accuracy: 1.0000 - val_loss: 4.8939e-04 - learning_rate: 0.0010
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 630ms/step - accuracy: 1.0000 - loss: 8.3997e-04 - val_accuracy: 1.0000 - val_loss: 5.0224e-05 - learning_rate: 0.0010
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 612ms/step - accuracy: 1.0000 - loss: 7.1285e-05 - val_accuracy: 1.0000 - val_loss: 7.8427e-06 - learning_rate: 0.0010
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 615ms/step - accuracy: 1.0000 - loss: 3.1332e-05 - val_accuracy: 1.0000 - val_loss: 1.6711e-06 - learning_rate: 0.0010
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 559ms/step - accuracy: 1.0000 - loss: 4.2142e-06 - val_accuracy: 1.0000 - val_loss: 4.4915e-07 - learning_rate: 0.0010
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step - accuracy: 1.000

#### Explicación de los parámetros

- **`monitor`**: La métrica que `ReduceLROnPlateau` va a observar. Generalmente, se usa `'val_loss'` para la pérdida en el conjunto de validación, pero puedes cambiarla a `'val_accuracy'` o cualquier otra métrica que estés monitoreando.

- **`factor`**: El factor de reducción de la tasa de aprendizaje. Por ejemplo, `0.5` reduce el valor de `learning_rate` a la mitad.

- **`patience`**: Número de épocas que el modelo entrenará sin mejora antes de reducir el `learning_rate`. Aquí, después de 5 épocas sin mejora, se reducirá la tasa de aprendizaje.

- **`min_lr`**: Valor mínimo al cual puede reducirse el `learning_rate`. Esto evita que se reduzca demasiado y se vuelva insignificante.

### **Evaluación y Ajuste del Modelo**

- **Pruebas en el Conjunto de Validación**: Evalúa el modelo en el conjunto de validación y calcula métricas como `mAP` para medir la precisión de detección.

- **Ajuste de Hiperparámetros y Fine-Tuning**: Si el modelo necesita mejorar, intenta ajustar hiperparámetros, activar más capas en el modelo base o aplicar técnicas de regularización.

### 9. **Guardar y Documentar el Modelo Entrenado**

- **Guardar el Modelo**: Almacena el modelo en un archivo `.h5` o en formato TensorFlow SavedModel.

- **Documentación**: Documenta el código y los parámetros utilizados para el entrenamiento. Incluye un archivo README que explique cómo usar el modelo para inferencias.

In [134]:
model.save('plasticBottle_detector.keras')

In [139]:
import numpy as np
import matplotlib.pyplot as plt

from keras._tf_keras.keras import models
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator


def probar_modelo(ruta_modelo, directorio_test, tamano_imagen=(300, 300), batch_size=32):
    """
    Función para cargar un modelo y evaluarlo en un conjunto de datos de imágenes.
    
    Parámetros:
    - ruta_modelo: str, ruta del modelo guardado en formato .keras.
    - directorio_test: str, ruta del directorio que contiene las imágenes de prueba organizadas en subcarpetas por clase.
    - tamano_imagen: tuple, tamaño de las imágenes que se le pasan al modelo (altura, ancho).
    - batch_size: int, número de imágenes que se procesan en un solo paso.
    """

    # Cargar el modelo desde el archivo .keras
    modelo = models.load_model(ruta_modelo)

    # Crear un generador de datos para cargar las imágenes de la carpeta
    datagen = ImageDataGenerator(rescale=1./255)  # Normaliza las imágenes

    # Cargar las imágenes desde el directorio de prueba
    generador_test = datagen.flow_from_directory(
        directorio_test,
        target_size=tamano_imagen,  # Ajustar tamaño
        batch_size=batch_size,
        class_mode='categorical',  # Cambiar a 'binary' si es clasificación binaria
        shuffle=False
    )

    # Evaluar el modelo con las imágenes de prueba
    pasos = generador_test.samples // generador_test.batch_size
    perdidas, precision = modelo.evaluate(generador_test, steps=pasos)

    print(f'Pérdida: {perdidas:.4f}')
    print(f'Precisión: {precision:.4f}')

    # Hacer predicciones con el modelo
    generador_test.reset()  # Reiniciar el generador para obtener imágenes desde el principio
    predicciones = modelo.predict(generador_test, steps=pasos)
    # Obtener nombres de las clases
    nombres_clases = list(generador_test.class_indices.keys())

    # Visualizar algunas predicciones
    # Cambiar tamaño de la figura para acomodar imágenes más pequeñas
    plt.figure(figsize=(10, 6))

    # Obtener un lote de imágenes y etiquetas
    for i in range(min(10, pasos)):  # Mostrar hasta 10 imágenes
        imagenes, etiquetas = generador_test.next()  # Obtener el siguiente lote
        for j in range(len(imagenes)):
            if j >= 10:  # Solo mostrar hasta 10 imágenes
                break
            plt.subplot(2, 5, i * batch_size + j + 1)  # Ajustar el subplot
            plt.imshow(imagenes[j])  # Mostrar la imagen
            plt.axis('off')

            # Obtener la clase predicha y la probabilidad
            clase_predicha = nombres_clases[np.argmax(
                predicciones[i * batch_size + j])]
            # Obtener la probabilidad más alta
            probabilidad = np.max(predicciones[i * batch_size + j])
            clase_real = nombres_clases[np.argmax(
                etiquetas[j])]  # Etiqueta real

            plt.title(f'Predicción: {clase_predicha} ({
                      probabilidad:.2f})\nReal: {clase_real}')

    plt.suptitle(f'Precisión del modelo: {precision:.4f}', fontsize=16)
    plt.tight_layout()
    plt.show()

    return modelo

In [140]:
# Ejemplo de uso
ruta_modelo = 'plasticBottle_detector.keras'  # Ruta donde guardaste tu modelo
# Ruta a tus imágenes de prueba
directorio_test = 'dataset/data/test/'
tamano_imagen = (300, 300)  # Tamaño esperado por tu modelo
batch_size = 32  # Tamaño del lote

# Llamar a la función para probar el modelo
probar_modelo(ruta_modelo, directorio_test, tamano_imagen, batch_size)

Found 4 images belonging to 1 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 3.8555e-09
Pérdida: 0.0000
Precisión: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 795ms/step


<Figure size 1000x600 with 0 Axes>

<Sequential name=sequential_5, built=True>